<a href="https://colab.research.google.com/github/VinaySingh561/LARGE-LANGUAGE-MODELS/blob/main/Shortcut_connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

In [2]:
class GELU(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,x):
    return 0.5 * x * (1 + torch.tanh(
        torch.sqrt(torch.tensor(2.0/torch.pi))*
        (x + 0.044715*torch.pow(x,3))
    ))


In [13]:
class DeepNeuralNetworkSkip(nn.Module):
  def __init__(self,layers_shape, shortcut_connection):
    super().__init__()
    self.shortcut_connection = shortcut_connection
    self.layers = nn.ModuleList([
    nn.Sequential(nn.Linear(layers_shape[0],layers_shape[1]),GELU()),
    nn.Sequential(nn.Linear(layers_shape[1],layers_shape[2]),GELU()),
    nn.Sequential(nn.Linear(layers_shape[2],layers_shape[3]),GELU()),
    nn.Sequential(nn.Linear(layers_shape[3],layers_shape[4]),GELU()),
    nn.Sequential(nn.Linear(layers_shape[4],layers_shape[5]),GELU())
    ])


  def forward(self,x):
    for layer in self.layers:
      layer_output = layer(x)
      if self.shortcut_connection and x.shape == layer_output.shape:
        x = layer_output + x
      else:
        x = layer_output
    return x


In [21]:
layer_sizes = [3,3,3,3,3,1]
sample_input = torch.tensor([[0.1,0.23,-1.49]])
torch.manual_seed(123)
model_without_shortcut = DeepNeuralNetworkSkip(layer_sizes,False)


In [22]:
def print_gradient(model,x):
  output = model(x)
  target  = torch.tensor([[3.9]])

  loss = nn.MSELoss()
  loss = loss(output, target)

  loss.backward()

  for name, param in model.named_parameters():
    if "weight" in name:
      print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")

In [23]:
print_gradient(model_without_shortcut,sample_input)

layers.0.0.weight has gradient mean of 0.014931912533938885
layers.1.0.weight has gradient mean of 0.019653478637337685
layers.2.0.weight has gradient mean of 0.06316769868135452
layers.3.0.weight has gradient mean of 0.07992622256278992
layers.4.0.weight has gradient mean of 0.2909848392009735


In [24]:
torch.manual_seed(123)
model_with_shortcut = DeepNeuralNetworkSkip(layer_sizes,True)

In [25]:
print_gradient(model_with_shortcut,sample_input)

layers.0.0.weight has gradient mean of 0.9692482352256775
layers.1.0.weight has gradient mean of 0.7377116680145264
layers.2.0.weight has gradient mean of 0.9747352600097656
layers.3.0.weight has gradient mean of 1.3277825117111206
layers.4.0.weight has gradient mean of 5.3807454109191895
